# Pre-Processing Agent

- This notebook helps you generate needed prompts for the LLM. 
- You need access to an LLM and execute the code provided by the LLM.

In [ ]:
import os
import io
import pandas as pd

def gather_summary(df):
    # 2. Capture df.info() output
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_text = buffer.getvalue()
    print(info_text)

    # 3. Calculate missing value stats
    missing_stats = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
    missing_summary = "\n".join([f"{col}: {val:.2f}%" for col, val in missing_stats.items()])
    print(missing_summary)

    # 4. Get column data types
    column_types = "\n".join([f"{col}: {dtype}" for col, dtype in df.dtypes.items()])
    print(column_types)

    # 5. Get unique value counts
    unique_counts = df.nunique()  # Will no longer fail on unhashable dict
    unique_counts_summary = "\n".join([f"{col}: {count}" for col, count in unique_counts.items()])
    print(unique_counts_summary)

    summary_text = f"""
    Dataset Name: {dataset_name}
    ----------------------------
    Shape: {df.shape[0]} rows x {df.shape[1]} columns
    
    Column Data Types:
    {column_types}
    
    Missing Value Percentage:
    {missing_summary}
    
    Unique Value Counts:
    {unique_counts_summary}
    
    Data (first {n_sample} rows):
    {df.head(n_sample).to_string()}
    
    Data Description:
    {df.describe().to_string()}
    
    Data Info:
    {info_text}
    """
    print(summary_text)

    return summary_text

In [2]:
dataset_name = "police_project.csv"

In [ ]:
n_sample=30
summary_text = ""
if os.path.exists(dataset_name):
    df = pd.read_csv(dataset_name)
    df = df.convert_dtypes()
    summary_text = gather_summary(df)

found file!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   stop_date           91741 non-null  string 
 1   stop_time           91741 non-null  string 
 2   county_name         0 non-null      Int64  
 3   driver_gender       86406 non-null  string 
 4   driver_age_raw      86414 non-null  Int64  
 5   driver_age          86120 non-null  Int64  
 6   driver_race         86408 non-null  string 
 7   violation_raw       86408 non-null  string 
 8   violation           86408 non-null  string 
 9   search_conducted    91741 non-null  boolean
 10  search_type         3196 non-null   string 
 11  stop_outcome        86408 non-null  string 
 12  is_arrested         86408 non-null  boolean
 13  stop_duration       86408 non-null  string 
 14  drugs_related_stop  91741 non-null  boolean
dtypes: Int64(3), boolean(3), string(9)
memory

# Data Cleaning Expert Agent	

- Collects dataset characteristics from your dataset
- Prompts LLM for which steps to perform based on the data characteristics


In [5]:
if summary_text:    
    prompt=f"""
    You are a Data Cleaning Expert. Given the following information about the data, 
    recommend a series of numbered steps to take to clean and preprocess it. 
    The steps should be tailored to the data characteristics and should be helpful 
    for a data cleaning agent that will be implemented.
    
    General Steps:
    Things that should be considered in the data cleaning steps:
    
    * Removing columns if more than 40 percent of the data is missing
    * Imputing missing values with the mean of the column if the column is numeric
    * Imputing missing values with the mode of the column if the column is categorical
    * Converting columns to the correct data type
    * Removing duplicate rows
    * Removing rows with missing values
    * Removing rows with extreme outliers (3X the interquartile range)
    
    Custom Steps:
    * Analyze the data to determine if any additional data cleaning steps are needed.
    * Recommend steps that are specific to the data provided. Include why these steps are necessary or beneficial.
    * If no additional steps are needed, simply state that no additional steps are required.
    
    IMPORTANT:
    Make sure to take into account any additional user instructions that may add, remove or modify some of these steps. Include comments in your code to explain your reasoning for each step. Include comments if something is not done because a user requested. Include comments if something is done because a user requested.
    
    Below are summaries of all datasets provided:
    {summary_text}
    
    Return steps as a numbered list. You can return short code snippets to demonstrate actions. But do not return a fully coded solution. The code will be generated separately by a Coding Agent.
    
    Avoid these:
    1. Do not include steps to save files.
    2. Do not include unrelated user instructions that are not related to the data cleaning.
    """

    print(prompt)
    


    You are a Data Cleaning Expert. Given the following information about the data, 
    recommend a series of numbered steps to take to clean and preprocess it. 
    The steps should be tailored to the data characteristics and should be helpful 
    for a data cleaning agent that will be implemented.
    
    General Steps:
    Things that should be considered in the data cleaning steps:
    
    * Removing columns if more than 40 percent of the data is missing
    * Imputing missing values with the mean of the column if the column is numeric
    * Imputing missing values with the mode of the column if the column is categorical
    * Converting columns to the correct data type
    * Removing duplicate rows
    * Removing rows with missing values
    * Removing rows with extreme outliers (3X the interquartile range)
    
    Custom Steps:
    * Analyze the data to determine if any additional data cleaning steps are needed.
    * Recommend steps that are specific to the data provided. 

Ask LLM for which steps to perform based on the data characteristics
Then, manually extract suggested steps and add them to the following list.

In [8]:
recommended_steps = [
    f"""
# Data Cleaning Steps for police_project.csv

## General Steps:

1. **Remove columns with more than 40% missing values:**
   - Remove `county_name` (100% missing)
   - Remove `search_type` (96.52% missing)

2. **Handle missing values in remaining columns:**
   - For numeric column `driver_age`: impute missing values with the mean
   - For categorical columns (`driver_gender`, `driver_race`, `violation_raw`, `violation`, `stop_outcome`, `stop_duration`): impute missing values with the mode
   - For `driver_age_raw`: Investigate values before deciding on imputation strategy

3. **Convert columns to correct data types:**
   - Convert `stop_date` and `stop_time` to datetime objects
   - Consider converting `driver_age_raw` to categorical if it represents birth years

4. **Remove duplicate rows:**
   - Identify and remove any duplicate rows in the dataset

5. **Remove rows with missing values:**
   - After imputation, remove any remaining rows with missing values

6. **Remove rows with extreme outliers:**
   - For `driver_age`: remove values beyond 3 times the interquartile range
   - For `driver_age_raw`: investigate and clean unrealistic values (0 to 8,801)

## Custom Steps:

7. **Check for consistency between `driver_age_raw` and `driver_age` columns:**
   - Verify if the calculated age from `driver_age_raw` matches the `driver_age` value
   - Resolve any inconsistencies between these columns

8. **Combine `stop_date` and `stop_time` into a single datetime column:**
   - Create a new column `stop_datetime` by combining date and time information
   - This will facilitate time-based analysis

9. **Convert `stop_duration` from string to numeric representation:**
   - Convert values like "0-15 Min" to 7.5, "16-30 Min" to 23, and "30+ Min" to 45
   - This will allow for quantitative analysis of stop duration

10. **Validate logical relationships between columns:**
    - Check if `is_arrested` is consistent with `stop_outcome`
    - Verify if `search_conducted` aligns with non-null values in `search_type`
"""  
]

In [9]:
function_name = "clean_data"

if summary_text and len(recommended_steps) > 0:
    steps_text = "\n".join(map(str, recommended_steps))
    
    prompt2=f"""You are a Data Cleaning Agent. Your job is to create a {function_name}() function that can be run on the data 
    provided using the following recommended steps.
    
    Recommended Steps:
    {steps_text}
    
    You can use Pandas, Numpy, and Scikit Learn libraries to clean the data.
    
    Below are summaries of all datasets provided. Use this information about the data to help determine how to 
    clean the data:
    
    {summary_text}
    
    Return Python code in ```python``` format with a single function definition, {function_name}(data_raw), that 
    includes all imports inside the function.
    
    Return code to provide the data cleaning function:
    
    def {function_name}(data_raw):
        import pandas as pd
        import numpy as np
        ...
        return data_cleaned
    
    Best Practices and Error Preventions:
    
    Always ensure that when assigning the output of fit_transform() from SimpleImputer to a Pandas DataFrame 
    column, you call .ravel() or flatten the array, because fit_transform() returns a 2D array while a DataFrame column is 1D
    """
        
    print(prompt2)

You are a Data Cleaning Agent. Your job is to create a clean_data() function that can be run on the data 
    provided using the following recommended steps.
    
    Recommended Steps:
    
# Data Cleaning Steps for police_project.csv

## General Steps:

1. **Remove columns with more than 40% missing values:**
   - Remove `county_name` (100% missing)
   - Remove `search_type` (96.52% missing)

2. **Handle missing values in remaining columns:**
   - For numeric column `driver_age`: impute missing values with the mean
   - For categorical columns (`driver_gender`, `driver_race`, `violation_raw`, `violation`, `stop_outcome`, `stop_duration`): impute missing values with the mode
   - For `driver_age_raw`: Investigate values before deciding on imputation strategy

3. **Convert columns to correct data types:**
   - Convert `stop_date` and `stop_time` to datetime objects
   - Consider converting `driver_age_raw` to categorical if it represents birth years

4. **Remove duplicate rows:**
   - 

## Fix Code Agent

In case of an error, provide stack-trace to LLM and re-run code

In [ ]:
code_snippet = f"""
    """
error = f"""
"""

if code_snippet and error:
    data_cleaner_prompt = f"""
            You are a Fix Code Agent. Your job is to create a {function_name}() function that can be run on the data provided. 
            The function is currently broken and needs to be fixed.
            
            Make sure to only return the function definition for {function_name}().
            
            Return Python code in ```python``` format with a single function definition, {function_name}(data_raw), 
            that includes all imports inside the function.
            
            This is the broken code (please fix): 
            {code_snippet}
    
            Last Known Error:
            {error}
            """
    print(data_cleaner_prompt)


            You are a Fix Code Agent. Your job is to create a clean_data() function that can be run on the data provided. 
            The function is currently broken and needs to be fixed.
            
            Make sure to only return the function definition for clean_data().
            
            Return Python code in ```python``` format with a single function definition, clean_data(data_raw), 
            that includes all imports inside the function.
            
            This is the broken code (please fix): 
            
# Step 8: Combine stop_date and stop_time into a single datetime column
    data['stop_datetime'] = data.apply(lambda x: pd.datetime.combine(x['stop_date'], x['stop_time']), axis=1)
    
    
            Last Known Error:
            
xception has occurred: AttributeError
module 'pandas' has no attribute 'datetime'
AttributeError: module 'pandas' has no attribute 'datetime'

            
